### Chapter 4: Control Flow Statements

Control flow statements in SystemVerilog allow you to control the execution path of your code based on conditions and loops. This chapter covers all essential control structures used in both synthesizable RTL design and testbench development.

#### if-else Statements

The `if-else` statement is the most fundamental conditional control structure in SystemVerilog.

##### Basic Syntax

```systemverilog
if (condition1) begin
    // statements
end else if (condition2) begin
    // statements
end else begin
    // statements
end
```

##### Single Statement (without begin-end)

```systemverilog
if (condition)
    statement;
else
    statement;
```

##### Practical Examples

###### Example 1: Simple Comparator
```systemverilog
module comparator(
    input logic [7:0] a, b,
    output logic gt, eq, lt
);
    always_comb begin
        if (a > b) begin
            gt = 1'b1;
            eq = 1'b0;
            lt = 1'b0;
        end else if (a == b) begin
            gt = 1'b0;
            eq = 1'b1;
            lt = 1'b0;
        end else begin
            gt = 1'b0;
            eq = 1'b0;
            lt = 1'b1;
        end
    end
endmodule
```

###### Example 2: Priority Encoder
```systemverilog
module priority_encoder(
    input logic [7:0] data_in,
    output logic [2:0] encoded_out,
    output logic valid
);
    always_comb begin
        if (data_in[7])
            encoded_out = 3'd7;
        else if (data_in[6])
            encoded_out = 3'd6;
        else if (data_in[5])
            encoded_out = 3'd5;
        else if (data_in[4])
            encoded_out = 3'd4;
        else if (data_in[3])
            encoded_out = 3'd3;
        else if (data_in[2])
            encoded_out = 3'd2;
        else if (data_in[1])
            encoded_out = 3'd1;
        else if (data_in[0])
            encoded_out = 3'd0;
        else
            encoded_out = 3'd0;
            
        valid = |data_in; // OR reduction
    end
endmodule
```

##### Best Practices for if-else
- Always use `begin-end` blocks for multiple statements
- Use `always_comb` for combinational logic
- Use `always_ff` for sequential logic
- Avoid complex nested conditions when possible

#### Case Statements

Case statements provide a cleaner alternative to multiple if-else statements when comparing a single expression against multiple values.

##### case Statement1

The standard `case` statement performs exact matching including X and Z values.

```systemverilog
case (expression)
    value1: statement1;
    value2: statement2;
    value3, value4: statement3; // Multiple values
    default: default_statement;
endcase
```

###### Example: ALU Design
```systemverilog
module alu(
    input logic [3:0] opcode,
    input logic [7:0] a, b,
    output logic [7:0] result,
    output logic zero
);
    always_comb begin
        case (opcode)
            4'b0000: result = a + b;        // ADD
            4'b0001: result = a - b;        // SUB
            4'b0010: result = a & b;        // AND
            4'b0011: result = a | b;        // OR
            4'b0100: result = a ^ b;        // XOR
            4'b0101: result = ~a;           // NOT
            4'b0110: result = a << 1;       // Shift left
            4'b0111: result = a >> 1;       // Shift right
            default: result = 8'h00;
        endcase
        
        zero = (result == 8'h00);
    end
endmodule
```

##### casex Statement

`casex` treats X and Z as don't-care values in both the case expression and case items.

```systemverilog
casex (data)
    4'b1???: // Matches any 4-bit value starting with 1
        result = "starts_with_1";
    4'b?1??: // Matches any 4-bit value with second bit as 1
        result = "second_bit_1";
    default:
        result = "other";
endcase
```

###### Example: Instruction Decoder
```systemverilog
module instruction_decoder(
    input logic [7:0] instruction,
    output logic [2:0] op_type
);
    always_comb begin
        casex (instruction)
            8'b000?????: op_type = 3'b001;  // Load instructions
            8'b001?????: op_type = 3'b010;  // Store instructions
            8'b010?????: op_type = 3'b011;  // Arithmetic
            8'b011?????: op_type = 3'b100;  // Logic
            8'b1???????: op_type = 3'b101;  // Branch
            default:     op_type = 3'b000;  // NOP
        endcase
    end
endmodule
```

##### casez Statement

`casez` treats only Z as don't-care values (more restrictive than casex).

```systemverilog
casez (selector)
    4'b1zzz: output = input1;
    4'bz1zz: output = input2;
    default: output = default_val;
endcase
```

##### Case Statement Guidelines
- Always include a `default` case
- Use `casex` for don't-care matching
- Use `casez` when only Z should be treated as don't-care
- Avoid overlapping case items

#### unique and priority Modifiers

SystemVerilog provides `unique` and `priority` modifiers to specify the intent and improve synthesis results.

##### unique Modifier

The `unique` modifier indicates that case items are mutually exclusive and exactly one will match.

```systemverilog
unique case (state)
    IDLE:  next_state = START;
    START: next_state = ACTIVE;
    ACTIVE: next_state = DONE;
    DONE:  next_state = IDLE;
endcase
```

##### priority Modifier

The `priority` modifier indicates that case items should be evaluated in order, and at least one will match.

```systemverilog
priority case (1'b1)
    error_flag:     status = ERROR;
    warning_flag:   status = WARNING;
    ready_flag:     status = READY;
    default:        status = IDLE;
endcase
```

##### Example: State Machine with unique
```systemverilog
typedef enum logic [1:0] {
    IDLE = 2'b00,
    READ = 2'b01,
    WRITE = 2'b10,
    DONE = 2'b11
} state_t;

module fsm(
    input logic clk, rst_n, start, rw,
    output logic busy, done
);
    state_t current_state, next_state;
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n)
            current_state <= IDLE;
        else
            current_state <= next_state;
    end
    
    always_comb begin
        unique case (current_state)
            IDLE: begin
                if (start)
                    next_state = rw ? WRITE : READ;
                else
                    next_state = IDLE;
            end
            READ: next_state = DONE;
            WRITE: next_state = DONE;
            DONE: next_state = IDLE;
        endcase
    end
    
    assign busy = (current_state != IDLE);
    assign done = (current_state == DONE);
endmodule
```

#### Loop Statements

SystemVerilog provides several loop constructs for different use cases.

##### for Loop

The `for` loop is used when the number of iterations is known.

```systemverilog
for (initialization; condition; increment) begin
    // statements
end
```

###### Example: Parallel-to-Serial Converter
```systemverilog
module parallel_to_serial(
    input logic clk, rst_n, load,
    input logic [7:0] parallel_in,
    output logic serial_out, done
);
    logic [7:0] shift_reg;
    logic [2:0] count;
    
    always_ff @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            shift_reg <= 8'h00;
            count <= 3'd0;
        end else if (load) begin
            shift_reg <= parallel_in;
            count <= 3'd0;
        end else if (count < 3'd7) begin
            shift_reg <= {shift_reg[6:0], 1'b0};
            count <= count + 1'b1;
        end
    end
    
    assign serial_out = shift_reg[7];
    assign done = (count == 3'd7);
endmodule
```

###### Example: Generate Loop for Parameterized Design
```systemverilog
module ripple_carry_adder #(parameter WIDTH = 8)(
    input logic [WIDTH-1:0] a, b,
    input logic cin,
    output logic [WIDTH-1:0] sum,
    output logic cout
);
    logic [WIDTH:0] carry;
    
    assign carry[0] = cin;
    
    generate
        for (genvar i = 0; i < WIDTH; i++) begin : adder_stage
            full_adder fa (
                .a(a[i]),
                .b(b[i]),
                .cin(carry[i]),
                .sum(sum[i]),
                .cout(carry[i+1])
            );
        end
    endgenerate
    
    assign cout = carry[WIDTH];
endmodule
```

##### while Loop

The `while` loop continues as long as the condition is true.

```systemverilog
while (condition) begin
    // statements
end
```

###### Example: Testbench with while Loop
```systemverilog
module tb_counter;
    logic clk, rst_n, enable;
    logic [3:0] count;
    integer test_cycles;
    
    counter dut (.*);
    
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    initial begin
        rst_n = 0;
        enable = 0;
        test_cycles = 0;
        
        #10 rst_n = 1;
        #10 enable = 1;
        
        while (test_cycles < 20) begin
            @(posedge clk);
            $display("Cycle %0d: count = %0d", test_cycles, count);
            test_cycles++;
        end
        
        $finish;
    end
endmodule
```

##### do-while Loop

The `do-while` loop executes at least once before checking the condition.

```systemverilog
do begin
    // statements
end while (condition);
```

###### Example: Random Test Generation
```systemverilog
class random_test;
    rand bit [7:0] data;
    
    function void generate_unique_values();
        bit [7:0] prev_value;
        
        do begin
            randomize();
        end while (data == prev_value);
        
        prev_value = data;
    endfunction
endclass
```

#### foreach Loops

The `foreach` loop iterates over arrays, providing a clean syntax for array operations.

```systemverilog
foreach (array_name[i]) begin
    // statements using array_name[i]
end
```

##### Example: Array Processing
```systemverilog
module array_processor;
    logic [7:0] data_array[16];
    logic [7:0] sum;
    integer i;
    
    initial begin
        // Initialize array
        foreach (data_array[i]) begin
            data_array[i] = i * 2;
        end
        
        // Calculate sum
        sum = 0;
        foreach (data_array[i]) begin
            sum += data_array[i];
        end
        
        $display("Array sum = %0d", sum);
    end
endmodule
```

##### Example: Multi-dimensional Array
```systemverilog
module matrix_operations;
    logic [7:0] matrix[4][4];
    logic [7:0] row_sum[4];
    
    initial begin
        // Initialize matrix
        foreach (matrix[i]) begin
            foreach (matrix[i][j]) begin
                matrix[i][j] = i + j;
            end
        end
        
        // Calculate row sums
        foreach (row_sum[i]) begin
            row_sum[i] = 0;
            foreach (matrix[i][j]) begin
                row_sum[i] += matrix[i][j];
            end
        end
        
        // Display results
        foreach (row_sum[i]) begin
            $display("Row %0d sum = %0d", i, row_sum[i]);
        end
    end
endmodule
```

#### repeat Statements

The `repeat` statement executes a block a specified number of times.

```systemverilog
repeat (expression) begin
    // statements
end
```

##### Example: Clock Generation
```systemverilog
module clock_generator;
    logic clk;
    
    initial begin
        clk = 0;
        
        repeat (100) begin
            #5 clk = ~clk;
            #5 clk = ~clk;
        end
        
        $display("Generated 100 clock cycles");
        $finish;
    end
endmodule
```

##### Example: Shift Register Test
```systemverilog
module shift_register_test;
    logic clk, rst_n, serial_in;
    logic [7:0] parallel_out;
    
    shift_register dut (.*);
    
    initial begin
        clk = 0;
        forever #5 clk = ~clk;
    end
    
    initial begin
        rst_n = 0;
        serial_in = 0;
        
        #10 rst_n = 1;
        
        // Shift in pattern 10110011
        repeat (8) begin
            @(posedge clk);
            serial_in = $random;
        end
        
        @(posedge clk);
        $display("Final parallel output: %b", parallel_out);
        $finish;
    end
endmodule
```

#### break and continue Statements

SystemVerilog supports `break` and `continue` statements for loop control.

##### break Statement

The `break` statement exits the innermost loop immediately.

```systemverilog
for (int i = 0; i < 100; i++) begin
    if (error_condition)
        break;
    // normal processing
end
```

##### continue Statement

The `continue` statement skips the rest of the current iteration and continues with the next iteration.

```systemverilog
for (int i = 0; i < 100; i++) begin
    if (skip_condition)
        continue;
    // processing for valid iterations
end
```

###### Example: Data Validation Loop
```systemverilog
module data_validator;
    logic [7:0] data_stream[100];
    logic [7:0] valid_data[$];
    
    initial begin
        // Initialize test data
        foreach (data_stream[i]) begin
            data_stream[i] = $random;
        end
        
        // Process data with validation
        foreach (data_stream[i]) begin
            // Skip invalid data (value 0 or 255)
            if (data_stream[i] == 0 || data_stream[i] == 255) begin
                $display("Skipping invalid data at index %0d: %0d", 
                        i, data_stream[i]);
                continue;
            end
            
            // Break on error pattern
            if (data_stream[i] == 8'hFF) begin
                $display("Error pattern detected at index %0d", i);
                break;
            end
            
            // Store valid data
            valid_data.push_back(data_stream[i]);
        end
        
        $display("Processed %0d valid data items", valid_data.size());
    end
endmodule
```

###### Example: Search Algorithm
```systemverilog
function int find_first_match(logic [7:0] array[], logic [7:0] target);
    foreach (array[i]) begin
        if (array[i] == target) begin
            return i;  // Found match, return index
        end
        
        // Skip processing for special values
        if (array[i] == 8'hXX) begin
            continue;
        end
        
        // Additional processing could go here
    end
    
    return -1; // Not found
endfunction
```

#### Best Practices and Guidelines

##### Control Flow Best Practices

1. **Use appropriate control structures**:
   - `if-else` for simple conditions
   - `case` for multi-way branching
   - `unique case` for mutually exclusive conditions
   - `priority case` for prioritized conditions

2. **Always include default cases**:
   ```systemverilog
   case (opcode)
       4'b0000: result = a + b;
       4'b0001: result = a - b;
       default: result = 8'h00;  // Always include
   endcase
   ```

3. **Use proper blocking assignments**:
   - Use `=` in `always_comb` blocks
   - Use `<=` in `always_ff` blocks

4. **Avoid complex nested conditions**:
   ```systemverilog
   // Instead of deeply nested if-else
   if (condition1) begin
       if (condition2) begin
           if (condition3) begin
               // deeply nested
           end
       end
   end
   
   // Use early returns or case statements
   case ({condition1, condition2, condition3})
       3'b111: // handle case
       3'b110: // handle case
       default: // handle default
   endcase
   ```

#### Synthesis Considerations

1. **Combinational vs Sequential Logic**:
   - Use `always_comb` for combinational logic
   - Use `always_ff` for sequential logic

2. **Avoid latches**:
   - Always assign values to all outputs in all branches
   - Use default assignments

3. **Resource implications**:
   - Complex case statements may require large multiplexers
   - Consider priority encoders for one-hot cases

#### Testbench Specific Guidelines

1. **Use unlimited loops carefully**:
   ```systemverilog
   // Good: bounded loop
   repeat (1000) @(posedge clk);
   
   // Risky: unlimited loop
   while (1) begin
       // ensure there's an exit condition
   end
   ```

2. **Use foreach for array iteration**:
   ```systemverilog
   // Preferred
   foreach (array[i]) begin
       process(array[i]);
   end
   
   // Less preferred
   for (int i = 0; i < array.size(); i++) begin
       process(array[i]);
   end
   ```

#### Summary

Control flow statements are fundamental to SystemVerilog design and verification. Key takeaways:

- **if-else statements** provide basic conditional execution
- **case statements** offer clean multi-way branching with variants (casex, casez)
- **unique and priority modifiers** specify design intent and improve synthesis
- **Loop statements** (for, while, do-while, foreach, repeat) handle iterative operations
- **break and continue** provide fine-grained loop control
- Proper use of control flow statements is crucial for both synthesizable RTL and testbench code

Understanding these control structures and their appropriate usage will enable you to write efficient, readable, and synthesizable SystemVerilog code.